In [4]:
import uproot
import pandas as pd
import mplhep as hep
import numpy as np
import awkward
import ROOT
import scipy.stats as stats #this one used to do fits
import matplotlib.pyplot as plt
import awkward as ak
import MyHelpers as mh
import StatisticTools as st
from tqdm import tqdm #this is a fancy feature to make a progress bar as the loop proceed


#to make the plots in CMS style execute this line
plt.style.use([hep.style.ROOT, hep.style.firamath])
plt.style.use(hep.style.CMS)

In [5]:
#Getting files
#Execute this line if running on SWAN, otherwise update the path to the data files:
PATH='/eos/cms/store/user/jjhollar/CERNSummerStudentProject2021/'
#PATH='data'
filename=PATH+'/gammagammaMuMu_FPMC_pT25_14TeV_PU200_NTUPLE_jobs123merge_version4.root'
#load the file content
tree = uproot.open(filename+":myana/mytree")
#tree.show()

filename=PATH+'DYMuMu_PU200_NTUPLE_1_version3.root'
bg_tree = uproot.open(filename+":myana/mytree")

In [6]:
#Create signal tree
event_info = tree.arrays(['genvtx_t0']) 
muons = tree.arrays(['pfcand_pt','pfcand_eta','pfcand_phi','pfcand_mass','pfcand_t','pfcand_vz'],'abs(pfcand_pid)==13') 
protons = tree.arrays(['genproton_xi','genproton_pz','genproton_vz','genproton_ispu'], '(genproton_ispu == 0) & (abs(genproton_pz)<6999) & (abs(genproton_pz)>0)')  
vertices = tree.arrays(['vtx4D_t','vtx4D_z','vtx4D_pt2']) 

#Create background tree
#Compressing the muon data
bg_muons =[]
for batch, report in bg_tree.iterate(['pfcand_pt','pfcand_eta','pfcand_phi','pfcand_mass','pfcand_t','pfcand_vz','pfcand_pid'], step_size=1000, report=True):
    #print(report)
    bg_muons.append(batch[(abs(batch.pfcand_pid)==13) & (batch.pfcand_pt>25)] )
bg_muons=ak.concatenate(bg_muons)

bg_protons = bg_tree.arrays(['genproton_xi','genproton_pz','genproton_vz','genproton_ispu'],'(abs(genproton_pz)<6999) & (abs(genproton_pz)>0)')
bg_event_info = bg_tree.arrays(['genvtx_t0']) 
bg_vertices = bg_tree.arrays(['vtx4D_t','vtx4D_z','vtx4D_pt2'])

In [8]:
s_t = []
s_vz = []
s_y = []
s_m = []

b_t = []
b_vz = []
b_y = []
b_m = []



sig_rate_20 = []
bg_rate_20 = []
sig_20 = []

sig_rate_50 = []
bg_rate_50 = []
sig_50 = []



for trail in tqdm(range(100)):
    res = 50
    sig, bg = st.create_frame(muons,protons,vertices, event_info, bg_muons,bg_protons, bg_vertices, bg_event_info)
    sigma_m = st.fit_mass(sig)
    sigma_y = st.fit_y(sig)
    sigma_vz = st.fit_vz(sig,res)
    sigma_t = st.fit_t(sig,res)
    number_acc_sig, number_full_sig, s_t, s_vz, s_y, s_m = st.full_selection(sig,res, sigma_m, sigma_y,sigma_vz,sigma_t)
    number_acc_bg, number_full_bg, b_t, b_vz, b_y, b_m = st.full_selection(bg,res, sigma_m, sigma_y,sigma_vz,sigma_t)
    sig_rate_50 = np.append(sig_rate_50, number_acc_sig/number_full_sig)
    bg_rate_50 = np.append(bg_rate_50, number_acc_bg/number_full_bg)
    sig_50 = np.append(sig_50,(number_acc_sig+number_acc_bg)/number_acc_sig)
    (mu, sigma) = stats.norm.fit(sig_50)
print("significance:",mu)
print("std:",sigma)
print("signal rates for 10 full selections at resolution 50:",sig_rate_50)
print("background rates for 10 full selections at resolution 50:",bg_rate_50)

for trail in tqdm(range(100)):
    res = 20
    sig, bg = st.create_frame(muons,protons,vertices, event_info, bg_muons,bg_protons, bg_vertices, bg_event_info)
    sigma_m = st.fit_mass(sig)
    sigma_y = st.fit_y(sig)
    sigma_vz = st.fit_vz(sig,res)
    sigma_t = st.fit_t(sig,res)
    number_acc_sig, number_full_sig, s_t, s_vz, s_y, s_m = st.full_selection(sig,res, sigma_m, sigma_y,sigma_vz,sigma_t)
    number_acc_bg, number_full_bg, b_t, b_vz, b_y, b_m = st.full_selection(bg,res, sigma_m, sigma_y,sigma_vz,sigma_t)
    sig_rate_20 = np.append(sig_rate_20, number_acc_sig/number_full_sig)
    bg_rate_20 = np.append(bg_rate_20, number_acc_bg/number_full_bg)    
    sig_20 = np.append(sig_20,(number_acc_sig+number_acc_bg)/number_acc_sig)
    (mu, sigma) = stats.norm.fit(sig_20)
print("significance:",mu)
print("std:", sigma)
print("signal rates for 10 full selections at resolution 20:",sig_rate_20)
print("background rates for 10 full selections at resolution 20:",bg_rate_20)

  0%|          | 0/100 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [5]:

(mu, sigma) = stats.norm.fit(sig_50)
print("significance for 50ps:")
print(mu)
print("std:", sigma)

(mu, sigma) = stats.norm.fit(sig_20)
print("significance for 20ps:")
print(mu)
print("std:", sigma)


significance for 50ps:
1.0
std: 0.0
significance for 20ps:
1.0
std: 0.0


In [ ]:
#trying to make it shorter...
for k in [s, b]:
    k+_t = []
    k+_vz = []
    k+_y = []
    k+_m = []
for s in ['sig', 'bg']:
    for r in ['20', '50']:
        s+'_rate_'+r = []
        'sig_'+r = []


for res in [20,50]:
    for trail in range(0,10):    
        sig, bg = st.create_frame(muons,protons,vertices, event_info, bg_muons,bg_protons, bg_vertices, bg_event_info)
        sigma_m = st.fit_mass(sig)
        sigma_y = st.fit_y(sig)
        sigma_vz = st.fit_vz(sig,res)
        sigma_t = st.fit_t(sig,res)
        number_acc_sig, number_full_sig, s_t, s_vz, s_y, s_m = st.full_selection(sig,res, sigma_m, sigma_y,sigma_vz,sigma_t)
        number_acc_bg, number_full_bg, b_t, b_vz, b_y, b_m = st.full_selection(bg,res, sigma_m, sigma_y,sigma_vz,sigma_t)
        sig_rate_+str(res) = np.append(sig_rate_+str(res), number_acc_sig/number_full_sig)
        bg_rate_+str(res) = np.append(bg_rate_+str(res), number_acc_bg/number_full_bg)
        sig_+str(res) = np.append(sig_+str(res),(number_acc_sig+number_acc_bg)/number_acc_sig)
    print("signal rates for 10 full selections at resolution"+str(res)+": ",sig_rate_+str(res))
    print("background rates for 10 full selections at resolution"+str(res)+": ",bg_rate_+str(res))
    (mu, sigma) = stats.norm.fit(sig_+str(res))
    print("significance:",mu)
    print("std:",sigma)